<!--NAVIGATION-->
< [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Contents](Index.ipynb) | [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) >

# 3.8 按列合并数据集： merge操作

Pandas的基本特性之一就是高性能的内存式数据连接（join）与合并（merge）操作。如果你有使用数据库的经验，那么对这类操作一定很熟悉。Pandas的主接口是``pd.merge``函数，下面让我们通过一些示例来介绍它的用法。

为了方便介绍，继续使用``display()``来显示结果。

In [168]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 3.8.1 关系代数

``pd.merge()``实现的功能基于关系代数（relational algebra）的一部分。关系代数是处理关系型数据的通用理论，绝大部分数据库的可用操作都以此为理论基础。关系代数方法论的强大之处在于，它提出的若干简单操作规则经过组合就可以为任意数据集构建十分复杂的操作。借助在数据库或程序里已经高效实现的基本操作规则，你可以完成许多非常复杂的操作。

Pandas在``pd.merge()``函数与``Series``和``Dataframe``的``join()``方法里实现了这些基本操作规则。下面来看看如何用这些简单的规则连接不同数据源的数据。

## 3.8.2 数据连接的类型

``pd.merge()``函数实现了三种数据连接的类型： 一对一、 多对一和多对多。这三种数据连接类型都通过``pd.merge()``接口进行调用，根据不同的数据连接需求进行不同的操作。下面将通过一些示例来演示这三种类型，并进一步介绍更多的细节。

### 1. 一对一连接

一对一连接可能是最简单的数据合并类型了，与3.7节介绍的按列合并十分相似。

如下面示例所示，有两个包含同一所公司员工不同信息的``DataFrames``：

In [169]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


若想将这两个``DataFrame``合并成一个``DataFrame``，可以用``pd.merge()``函数实现：

In [170]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


``pd.merge()``方法会发现两个``DataFrame``都有“employee”列，并会自动以这列作为键进行连接。两个输入的合并结果是一个新的``DataFrame``。

需要注意的是，**共同列的位置可以是不一致的**。例如在这个例子中，虽然``df1``与``df2``中“employee”列的位置是不一样的，但是``pd.merge()``函数会正确处理这个问题。

另外还需要注意的是，``pd.merge()``会默认**丢弃原来的行索引**，不过也可以自定义（详情请参见3.8.3节）。

### 2. 多对一连接


多对一连接是指，在需要连接的两个列中，有一列的值**有重复**。通过多对一连接获得的结果``DataFrame``将会保留重复值。请看下面的例子：

In [171]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

在结果``DataFrame``中多了一个“supervisor”列，里面有些值会因为输入数据的对应关系而有所重复。

### 3. 多对多连接

多对多连接是个有点儿复杂的概念，不过也可以理解。如果左右两个输入的共同列都包含重复值，那么合并的结果就是一种多对多连接。用一个例子来演示可能更容易理解。来看下面的例子，里面有一个``DataFrame``显示不同岗位人员的一种或多种能力。

通过多对多链接，就可以得知每位员工所具备的能力：

In [172]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


这三种数据连接类型可以直接与其他Pandas工具组合使用，从而实现各种各样的功能。但是工作中的真实数据集往往并不像示例中演示的那么干净、整洁。下面就来介绍``pd.merge()``的一些功能，它们可以让你更好地应对数据连接中的问题。

## 3.8.3 设置数据合并的键

我们已经见过``pd.merge()``的默认行为：它会将两个输入的一个或多个共同列作为键进行合并。但由于两个输入要合并的列通常都**不是同名的**，因此``pd.merge()``提供了一些参数处理这个问题。

### 1. 参数``on``的用法

最简单的方法就是直接将参数``on``设置为一个列名字符串或者一个包含多列名称的列表：

In [173]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

这个参数只能在两个``DataFrame``有共同列名的时候才可以使用。

### 2. ``left_on``与``right_on``参数

有时你也需要合并两个列名不同的数据集，例如前面的员工信息表中有一个字段不是“employee”而是“name”。在这种情况下，就可以用``left_on``和``right_on``参数来指定列名：

In [174]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

获取的结果中会有一个多余的列，可以通过``DataFrame``的``drop()``方法将这列去掉：

In [175]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### 3. ``left_index``与``right_index``参数

除了合并列之外，你可能还需要合并索引。就像下面例子中的数据那样：

In [176]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


你可以通过设置``pd.merge()``中的``left_index``和（或）``right_index``参数将索引设置为键来实现合并：

In [177]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


``left_index``、``right_index``：设置索引是否作为连接的键，通常``left_on=??``和``right_index=True``,``right_on=??``和``left_index=True``，或者``left_index=True``和``right_index=True``。本例中的索引已经通过``set_index``由0,1,2,3设置为Bob,Jake,Lisa，Sue。

为了方便，``DataFrame``实现了``join()``方法，它可以按照索引进行数据合并：

In [178]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


如果想将索引与列混合使用，那么可以通过结合``left_index``与``right_on``，或者结合``left_on``与``right_index``来实现：

In [179]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

当然，这些参数都适用于多个索引和（或）多个列名，函数接口非常简单。若想了解Pandas数据合并的更多信息，请参考 Pandas 文档中“Merge, Join, and Concatenate”部分。（http://pandas.pydata.org/pandas-docs/stable/merging.html)

## 3.8.4 设置数据连接的集合操作规则

通过前面的示例，我们总结出数据连接的一个重要条件：集合操作规则。当一个值出现在一个键列（key column），却没有出现在另一键列时，就需要考虑集合操作规则了。来看看下面的例子：

In [180]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

我们合并两个数据集，在“name”列中只有一个共同的值：Mary。默认情况下，结果中只会包含两个输入集合的**交集**，这种连接方式被称为**内连接（inner join）**。我们可以用``how``参数设置连接方式，默认值为``"inner"``：

In [181]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


``how``参数支持的数据连接方式还有``'outer'``、``'left'``和``'right'``。 **外连接（outer join）**返回两个输入列的并集，所有缺失值都用``NaN``填充：

In [182]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

**左连接（left join）**和**右连接（right join）**返回的结果分别只包含左列和右列，如下所示：

In [183]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

现在输出的行中只包含左边输入列的值。如果用``how='right'``的话，输出的行则只包含右边输入列的值。

这四种数据连接的集合操作规则都可以直接应用于前面介绍过的连接类型。

## 3.8.5 重复列名：``suffixes``参数

最后，你可能会遇到两个输入``DataFrame``有重名列的情况。来看看下面的例子：

In [184]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

由于输出结果中有两个重复的列名，因此``pd.merge()``函数会自动为它们增加后缀``_x``或``_y``，当然也可以通过``suffixes``参数自定义后缀名：

In [185]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

``suffixes``参数同样适用于任何连接方式，即使有三个及三个以上的重复列名时也同样适用。

## 3.8.6 案例：美国各州的统计数据

数据的合并与连接是组合来源不同的数据的最常用方法。下面通过美国各州的统计数据来进行一个演示,主要需求是合并三个DataFrame，求美国各州的人口密度，但由于不同表格里的同一个含义的两列列名不同、有的行里含有缺失值、有的行不需要，我们需要不断地应用Pandas的方法去查看数据，并执行批量的处理（使用到的方法有：merge, isnull及其掩码操作与花哨索引, any, unique, dropna, sort_values, head, tail等：

用Pandas的``read_csv()``函数引入三个数据集：

In [186]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

In [187]:
help(pop.head) #查看head方法的帮助

Help on method head in module pandas.core.generic:

head(n=5) method of pandas.core.frame.DataFrame instance
    Return the first `n` rows.
    
    This function returns the first `n` rows for the object based
    on position. It is useful for quickly testing if your object
    has the right type of data in it.
    
    Parameters
    ----------
    n : int, default 5
        Number of rows to select.
    
    Returns
    -------
    obj_head : same type as caller
        The first `n` rows of the caller object.
    
    See Also
    --------
    DataFrame.tail: Returns the last `n` rows.
    
    Examples
    --------
    >>> df = pd.DataFrame({'animal':['alligator', 'bee', 'falcon', 'lion',
    ...                    'monkey', 'parrot', 'shark', 'whale', 'zebra']})
    >>> df
          animal
    0  alligator
    1        bee
    2     falcon
    3       lion
    4     monkey
    5     parrot
    6      shark
    7      whale
    8      zebra
    
    Viewing the first 5 lines
    


看过这些数据之后，我们想要计算一个比较简单的指标：美国各州的人口密度排名。虽然可以直接通过计算每张表获取结果，但这次试着用**数据集连接**来解决这个问题。

首先以**多对一**方式合并获取人口（pop）``DataFrame``中各州名称缩写对应的全称。我们需要将``pop``的``state/region``列与``abbrevs``的``abbreviation``列进行合并，还需要通过``how='outer'``确保数据没有丢失（请help(pd.merge)查看详细说明）。

In [188]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
#回顾：left_on、right_on：在连接的键名不同的情况下使用，left_on传入左对象的键，right_on传入右对象的键
merged = merged.drop('abbreviation',axis=1) #丢弃重复信息
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


来全面检查一下数据是否有缺失，我们可以对每个字段逐行检查是否有缺失值：

In [189]:
type(merged.isnull())

pandas.core.frame.DataFrame

In [190]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

建议：上行代码的用法可以分解去看：请print(merged.isnull())，使用help(merged.isnull)和help(merged.isnull().any())查看帮助。

部分``population``是缺失值，让我们仔细看看那些数据！

In [191]:
merged[merged['population'].isnull()].head()#此处用到掩码操作，见02.06

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


所有的人口缺失值都出现在PR。

同时，我们还发现一些新的``state``的数据也有缺失，可能是由于``abbrevs``键没有匹配上全称！来看看究竟是哪个州有缺失：

In [192]:
merged.loc[merged['state'].isnull(), 'state/region']##此处用到花哨的索引，见02.07
#请试试merged.loc[merged['state'].isnull(), 'state/region'].unique(),你会更明确的得到PR和USA

2448     PR
2449     PR
2450     PR
2451     PR
2452     PR
2453     PR
2454     PR
2455     PR
2456     PR
2457     PR
2458     PR
2459     PR
2460     PR
2461     PR
2462     PR
2463     PR
2464     PR
2465     PR
2466     PR
2467     PR
2468     PR
2469     PR
2470     PR
2471     PR
2472     PR
2473     PR
2474     PR
2475     PR
2476     PR
2477     PR
       ... 
2514    USA
2515    USA
2516    USA
2517    USA
2518    USA
2519    USA
2520    USA
2521    USA
2522    USA
2523    USA
2524    USA
2525    USA
2526    USA
2527    USA
2528    USA
2529    USA
2530    USA
2531    USA
2532    USA
2533    USA
2534    USA
2535    USA
2536    USA
2537    USA
2538    USA
2539    USA
2540    USA
2541    USA
2542    USA
2543    USA
Name: state/region, Length: 96, dtype: object

我们可以快速解决这个问题：人口数据中包含波多黎各（PR）和全国总数（USA），但这两项没有出现在州名称缩写表中。来快速填充对应的全称。

In [193]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

现在``state``列没有缺失值了，万事俱备！

让我们用类似的规则将面积数据也合并进来。用两个数据集共同的``state``列来合并：

In [194]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


再检查一下数据，看看哪些列还有缺失值，没有匹配上：

In [195]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

面积``area``列里面还有缺失值。来看看究竟是哪些地区面积缺失：

In [196]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

我们发现面积``areas``在 ``DataFrame``里面不包含全美国的面积数据。可以插入全国总面积数据（对各州面积求和即可），但是针对本案例，我们要去掉这个缺失值，因为全国的人口密度在此无关紧要：

In [197]:
final.dropna(inplace=True)#inplace:原地，这里表示原地(原数据)修改
final.head()
#Pandas 中 inplace 参数在很多函数中都会有，它的作用是：是否在原对象基础上进行修改。
#inplace = True：不创建新的对象，直接对原始对象进行修改。
#inplace = False：创建并返回新的对象承载其修改结果。

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


#现在所有的数据都准备好了。为了解决眼前的问题，先选择2010年的各州人口以及总人口数据。让我们用``query()``函数进行快速计算（这需要用到``numexpr``程序库，详情请参见3.13 节）：

In [198]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


现在来计算人口密度并按序排列。首先对索引进行重置，然后再计算结果：

In [199]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
#type(density)发现它是一个Series对象

In [200]:
density.sort_values(ascending=False, inplace=True) #ascending=False表示不进行升序排列，即降序排列
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

计算结果是美国各州加上华盛顿特区（Washington, DC）、波多黎各在 2010 年的人口密度排序，以万人/平方英里为单位。我们发现人口密度最高的地区是华盛顿特区的哥伦比亚地区（the District of Columbia）。在各州的人口密度中，新泽西州（New Jersey）是最高的。

还可以看看人口密度最低的几个州的数据：

In [201]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

可以看出，人口密度最低的州是阿拉斯加（Alaska），刚刚超过 1 万人/平方英里。

当人们用现实世界的数据解决问题时，合并这类脏乱的数据是十分常见的任务。希望这个案例可以帮你把前面介绍过的工具串起来，从而在数据中找到想要的答案！

<!--NAVIGATION-->
< [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) | [Contents](Index.ipynb) | [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) >